# Diplodatos Kaggle Competition

We present this peace of code to create the baseline for the competition, and as an example of how to deal with these kind of problems. The main goals are that you:

1. Learn
1. Try different models and see which one fits the best the given data
1. Get a higher score than the given one in the current baseline example
1. Try to get the highest score in the class :)

In [ ]:
# Import the required packages
import os
import warnings

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import ensemble
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier as DT
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing


Read the *original* dataset...

In [ ]:
from hashlib import md5
def hashit(val):
    if isinstance(val, float): 
        return str(val)
    return md5(val.encode('utf-8')).hexdigest()



In [ ]:
def get_dia_laboral(nombre_dia):
    if nombre_dia in ['Sunday', 'Saturday']: #['Wednesday', 'Thursday', 'Friday', 'Monday','Tuesday']:
        #return 'Fin de semana'
        return 1
    else:
        #return 'Dia laboral'
        return 0

In [ ]:
def transform_data(train_data_fname, test_data_fname):
    df_train = pd.read_csv(train_data_fname)
    df_train['is_train_set'] = 1
    df_test = pd.read_csv(test_data_fname)
    df_test['is_train_set'] = 0

##################################################################
### NUEVO VERSION 3
##################################################################
    df_train['W']=df_train.Weekday.apply(lambda x:get_dia_laboral(x))
    df_train.drop('Weekday', axis=1, inplace=True)
    df_train.rename(columns={'W': 'Weekday'}, inplace=True)
    #display(df_train.head(20))
########
    df_test['W']=df_test.Weekday.apply(lambda x:get_dia_laboral(x))
    df_test.drop('Weekday', axis=1, inplace=True)
    df_test.rename(columns={'W': 'Weekday'}, inplace=True)
    #display(df_test.head(20))
##################################################################

    # we  get the TripType for the train set. To do that, we group by VisitNumber and
    # then we get the max (or min or avg)
    ######### PROBAR CAMBIANDO A MIN Y AVG...#################################
    y = df_train.groupby(["VisitNumber", "Weekday"], as_index=False).max().TripType
    ##display(y)
    # we remove the TripType now, and concat training and testing data
    # the concat is done so that we have the same columns for both datasets
    # after one-hot encoding
    df_train = df_train.drop("TripType", axis=1)
    df = pd.concat([df_train, df_test])
    ##display(df.head(20))
    # the next three operations are the ones we have just presented in the previous lines
### NUEVO VERSION 3###############
#### ACTIVO EL BORRADO   
    df.drop_duplicates(keep='first', ignore_index=True, inplace=True)
#########################

    # drop the columns we won't use (it may be good to use them somehow)
    #df = df.drop(["Upc"], axis=1)
    #################################################
    ### NUEVO MELISA
    #################################################
    mask = df.Upc.isna()
    column_name = 'Upc'
    df.loc[mask, column_name] = 0
    #################################################

    mask = (df.FinelineNumber.isna())&(df.DepartmentDescription=='PHARMACY RX')
    column_name = 'FinelineNumber'
    df.loc[mask, column_name] = 4822.0


    # one-hot encoding for the DepartmentDescription
### DESACTIVO Y CAMBIO POR LABEL ENCODER...
    df = pd.get_dummies(df, columns=["DepartmentDescription"], dummy_na=True)
####################################################################################################


### NUEVO ###########################################################################################
### NO SIRVIÓ, LO APAGO...Y REACTIVO PANDAS DUMMIES
# Create a label (category) encoder object
    #le = preprocessing.LabelEncoder()

# Fit the encoder to the pandas column
    #le.fit(df['DepartmentDescription'].dropna())
    #df['D'] = df['DepartmentDescription'].map(lambda x: le.transform([x])[0] if type(x)==str else x) 

    #df=df.drop(["DepartmentDescription"], axis=1)
    #df.rename(columns={'D': 'DepartmentDescription'}, inplace=True)
#####################################################################################################

    # now we add the groupby values
    #df = df.groupby(["VisitNumber", "Weekday","FinelineNumber"], as_index=False).sum()
    df = df.groupby(["VisitNumber", "Weekday"], as_index=False).sum()

#### NUEVO - DESACTIVO ESTO... #################################################
#    df['tipo_dia']=df.Weekday.apply(lambda x:get_dia_laboral(x))
    df = pd.get_dummies(df, columns=["Weekday"], dummy_na=True)

#    # finally, we do one-hot encoding for the Weekday
#    df = pd.get_dummies(df, columns=["Weekday"], dummy_na=True)
################################################################################

    # get train and test back
    df_train = df[df.is_train_set != 0]
    df_test = df[df.is_train_set == 0]
    
    X = df_train.drop(["is_train_set"], axis=1)
    yy = None
    XX = df_test.drop(["is_train_set"], axis=1)

    return X, y, XX, yy

Load the data...

In [ ]:
X, y, XX, yy = transform_data("https://raw.githubusercontent.com/DiploDatos/AprendizajeSupervisado/master/practico/data/train.csv", "https://raw.githubusercontent.com/DiploDatos/AprendizajeSupervisado/master/practico/data/test.csv")

Create the model and evaluate it

In [ ]:
X.shape

(67029, 76)

In [ ]:
# split training dataset into train and "validation" 
# (we won't be using validation set in this example, because of the cross-validation;
# but it could be useful for you depending on your approach)
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
# results dataframe is used to store the computed results
results = pd.DataFrame(columns=('clf', 'best_acc'))

In [ ]:
################################################
### NUEVO VERSION 4
################################################
#ASI CORRIÓ TODA LA NOCHE Y NO TERMINÓ
#parameters = {
#    'n_estimators'      : [50, 75, 100, 120, 149],
#    'random_state'      : [2],
#    'max_features': ['auto','sqrt','log2'],
#    'criterion' :['gini','entropy'],
#    'max_depth' : [5,7,9,11,None],
#    'min_samples_split': [2, 3, 5, 7],
#    'min_samples_leaf': [1, 3, 5, 7],
#    'max_leaf_nodes':  [2, 3, 4, 5, 20, 50, 69],
#    'oob_score': [True, False],
#}

######### HAGO UN PEQUEÑO CAMBIO... SINO, LO DEJO COMO ESTABA....###################

##################################################################################
#parameters = {
#    'randomforestclassifier__n_estimators'      : [50, 100, 120, 137, 150],
#    'randomforestclassifier__random_state'      : [2],
#    'randomforestclassifier__max_features': ['auto','sqrt'],
#    'randomforestclassifier__criterion' :['gini','entropy'],
#    'randomforestclassifier__max_depth' : [5,7,9,11,None],
#    'randomforestclassifier__min_samples_split': [2, 3, 5, 7],
#    'randomforestclassifier__min_samples_leaf': [1, 3, 5, 7],
#    'randomforestclassifier__max_leaf_nodes':  [2, 3, 5, 7],
#    'randomforestclassifier__oob_score': [True, False],
#}

##################################################################
### VERSIÓN 2 y 3
##################################################################

parameters = {
    'randomforestclassifier__n_estimators' : [50,100,120],
    'randomforestclassifier__random_state' : [2],
    'randomforestclassifier__max_features' : ['auto'],
    'randomforestclassifier__criterion'    : ['gini','entropy']
}

with warnings.catch_warnings():
    warnings.filterwarnings("ignore")
    model=make_pipeline(StandardScaler(), RandomForestClassifier(random_state=0))
    randomtree_clf = GridSearchCV(model, parameters, scoring='accuracy', cv=3)
    randomtree_clf.fit(X_train, y_train)
    best_tree_clf = randomtree_clf.best_estimator_

randomtree_clf

GridSearchCV(cv=3, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('standardscaler',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('randomforestclassifier',
                                        RandomForestClassifier(bootstrap=True,
                                                               ccp_alpha=0.0,
                                                               class_weight=None,
                                                               criterion='gini',
                                                               max_depth=None,
                                                               max_features='auto',
                                                               max_leaf_nodes=None,
                   

In [ ]:
best_tree_clf

Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('randomforestclassifier',
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=None, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=1, min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators=120, n_jobs=None,
                                        oob_score=False, random_state=2,
                                        verbose=0, warm_start=False))],
         verbose=Fals

In [ ]:
print('Best Decision Tree accuracy: ', randomtree_clf.best_score_)
print(randomtree_clf)
results = results.append({'clf': randomtree_clf, 'best_acc': randomtree_clf.best_score_}, ignore_index=True)

print('The best classifier so far is: ')
print(results.loc[results['best_acc'].idxmax()]['clf'])

#Kaggle_5=Best Decision Tree accuracy:  0.6910743768773249
#The best classifier so far is: 
#GridSearchCV(cv=3, error_score=nan,
#             estimator=Pipeline(memory=None,
#                                steps=[('standardscaler',
#                                        StandardScaler(copy=True,
#                                                       with_mean=True,
#                                                       with_std=True)),
#                                       ('randomforestclassifier',
#                                        RandomForestClassifier(bootstrap=True,
#                                                               ccp_alpha=0.0,
#                                                               class_weight=None,
#                                                               criterion='gini',
#                                                               max_depth=None,
#                                                               max_features='auto',
#                                                               max_leaf_nodes=None,
#                                                               max_samples=None,
#                                                               min_impurity_decrea...
#                                                               warm_start=False))],
#                                verbose=False),
#             iid='deprecated', n_jobs=None,
#             param_grid={'randomforestclassifier__criterion': ['gini',
#                                                               'entropy'],
#                         'randomforestclassifier__max_features': ['auto'],
#                         'randomforestclassifier__n_estimators': [50, 100, 120],
#                         'randomforestclassifier__random_state': [2]},
#             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
#             scoring='accuracy', verbose=0)

Best Decision Tree accuracy:  0.6910743768773249
GridSearchCV(cv=3, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('standardscaler',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('randomforestclassifier',
                                        RandomForestClassifier(bootstrap=True,
                                                               ccp_alpha=0.0,
                                                               class_weight=None,
                                                               criterion='gini',
                                                               max_depth=None,
                                                               max_features='auto',
                                                      

In [ ]:
predictions = randomtree_clf.predict(X_valid)
print ('Accuracy: %d ' % ((np.sum(y_valid == predictions))/float(y_valid.size)*100))

Accuracy: 69 


In [ ]:
yy = randomtree_clf.predict(XX)

Exportamos Resultados


In [ ]:
submission5 = pd.DataFrame(list(zip(XX.VisitNumber, yy)), columns=["VisitNumber", "TripType"])

In [ ]:
submission5.to_csv("sample_data/submission_randomforest5.csv", header=True, index=False)